# Urban Heat Island (UHI) Notebook 

## Load In Dependencies

In [1]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
import geopandas as gpd

# Geospatial operations
import rasterio
from rasterio import windows  
from rasterio import features  
from rasterio import warp
from rasterio.warp import transform_bounds 
from rasterio.windows import from_bounds 

# Image Processing
from PIL import Image

# Coordinate transformations
from pyproj import Proj, Transformer, CRS

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.feature_selection import mutual_info_regression


# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm



## Response Variable

Before building the model, we need to load in the Urban Heat Island (UHI) index training dataset. We have curated data for the New York region. The dataset consists of geo-locations (Longitude and Latitude), with additional fields including date & time of data collection and the UHI index for each location. 

In [3]:
# Load the training data from csv file and display the first few rows to inspect the data
ground_df = pd.read_csv("Training_data_uhi_index_weather.csv")
ground_df.head()

,Longitude,Latitude,datetime,UHI Index,location,rounded_time,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,-73.919037,40.814292,2021-07-24 15:53:00,1.034616,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
1,-73.918978,40.814365,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
2,-73.918927,40.814433,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
3,-73.918875,40.814500,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
4,-73.918827,40.814560,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621


## Predictor Variables

<p align="justify">Gather the predictor variables from the Sentinel-2 dataset. Sentinel-2 optical data provides high-resolution imagery that is sensitive to land surface characteristics, which are crucial for understanding urban heat dynamics. For a more in-depth look regarding the Sentinel-2 dataset and how to query it, see the Microsoft Planetary Computer example <a href="https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a#Example-Notebook">Sentinel-2 supplementary notebook</a>. </p>


### Analyze the Sentinel-2 Data

<p align="justify">To obtain the Sentinel-2 data, we created a GeoTIFF image for a specific date and area of interest, which in this case is the Bronx and Manhattan regions of New York. The GeoTIFF product allows us to extract the relevant band values.

### Methods of Extracting Band Values from Sentinel-2 Data

There are two common methods to extract band values from Sentinel-2 data:

<ul>
    <li><strong>Using API Calls:</strong> Retrieve band values directly from Sentinel-2 datasets via APIs, such as the <code>planetary_computer</code>.</li>
</ul>
<ul>
    <li><strong>Using GeoTIFF Images:</strong> Create and download a GeoTIFF image containing the desired bands and extract the band values locally. The GeoTIFF image can represent any desired time period (single date or time series mosaic) and include any number of spectral bands.</li>
</ul>

Since our dataset is large, the API method can be time-consuming and resource-intensive. Therefore, in this sample notebook, we have opted for the second method.

### Downloading GeoTIFF Image

For building a sample model in this demonstration notebook, we have downloaded a GeoTIFF file locally for a single day (24th July 2021). The file is named <b>S2_sample.tiff</b>. In the subsequent section, we will use this GeoTIFF file to extract the band values for the geo-locations given in the training dataset to create the features.

First, let’s visualize the bands of the downloaded GeoTIFF image.




### Extracting Band Values from the GeoTIFF Image


In [5]:
def map_satellite_data(s2_tiff_path, lst_tiff_path, csv_path):
    """
    Extracts satellite band values from Sentinel-2 and Landsat LST GeoTIFFs based on coordinates from a CSV file.

    Parameters:
    s2_tiff_path (str): Path to the Sentinel-2 GeoTIFF file.
    lst_tiff_path (str): Path to the Landsat LST GeoTIFF file.
    csv_path (str): Path to the CSV file containing latitude and longitude coordinates.

    Returns:
    pd.DataFrame: A DataFrame containing extracted band values for each coordinate.
    """
    
    # Load the Sentinel-2 GeoTIFF data
    s2_data = rxr.open_rasterio(s2_tiff_path)
    s2_crs = s2_data.rio.crs  # Sentinel-2's Coordinate Reference System (CRS)

    # Load the Landsat LST GeoTIFF data
    lst_data = rxr.open_rasterio(lst_tiff_path)
    lst_crs = lst_data.rio.crs  # Landsat LST's Coordinate Reference System (CRS)

    # Read the CSV file
    df = pd.read_csv(csv_path)
    latitudes, longitudes = df['Latitude'].values, df['Longitude'].values

    # Setup CRS transformation (WGS84 -> Sentinel-2 CRS)
    s2_transformer = Transformer.from_crs("EPSG:4326", s2_crs, always_xy=True)

    # Setup CRS transformation (WGS84 -> Landsat LST CRS)
    lst_transformer = Transformer.from_crs("EPSG:4326", lst_crs, always_xy=True)

    # Transform lat/lon to the Sentinel-2 and Landsat LST coordinate systems
    s2_x_coords, s2_y_coords = s2_transformer.transform(longitudes, latitudes)
    lst_x_coords, lst_y_coords = lst_transformer.transform(longitudes, latitudes)

    # Define the Sentinel-2 bands
    s2_band_names = ["B01", "B02", "B03", "B04", "B06", "B08", "B11", "B12"]
    band_values = {band: [] for band in s2_band_names}
    band_values["lwir11"] = []  # Add Landsat LST band

    # Single iteration over coordinates
    for x_s2, y_s2, x_lst, y_lst in tqdm(zip(s2_x_coords, s2_y_coords, lst_x_coords, lst_y_coords), total=len(s2_x_coords), desc="Extracting band values"):
        # Extract Sentinel-2 band values
        for band_index, band in enumerate(s2_band_names, start=1):
            value = s2_data.sel(x=x_s2, y=y_s2, band=band_index, method="nearest").values
            band_values[band].append(value)

        #Extract Landsat LST band value
        value = lst_data.sel(x=x_lst, y=y_lst, method="nearest").values
        band_values["lwir11"].append(float(value))

    # Create a DataFrame with extracted values
    result_df = pd.DataFrame(band_values)
    
    return result_df

In [6]:
# Mapping satellite data with training data.
final_data = map_satellite_data('S2_sample.tiff', 'Landsat_LST.tiff', 'Training_data_uhi_index.csv')

Extracting band values: 100%|██████████| 11269/11269 [00:34<00:00, 324.52it/s]


In [7]:
final_data.head()

,B01,B02,B03,B04,B06,B08,B11,B12,lwir11
0,1561.0,1390.0,1646.0,1686.0,1831.0,1854.0,2296.0,2194.0,38.681055
1,1561.0,1390.0,1646.0,1686.0,1831.0,1854.0,2296.0,2194.0,38.681055
2,1502.0,1214.0,1184.0,1408.0,1755.0,1510.0,2168.0,2089.0,38.681055
3,1566.0,1286.0,1570.0,1656.0,2169.0,1832.0,2319.0,2022.0,38.687891
4,1566.0,1338.0,1390.0,1492.0,1648.0,1810.0,1890.0,1659.0,38.404195


## Joining the predictor variables and response variables
Now that we have extracted our predictor variables, we need to join them onto the response variable . We use the function <i><b>combine_two_datasets</b></i> to combine the predictor variables and response variables.The <i><b>concat</b></i> function from pandas comes in handy here.

In [8]:
# Combine two datasets vertically (along columns) using pandas concat function.
def combine_two_datasets(dataset1,dataset2):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    
    data = pd.concat([dataset1,dataset2], axis=1)
    return data

In [9]:
# Combining ground data and final data into a single dataset.
uhi_data = combine_two_datasets(ground_df,final_data)
uhi_data.head()

,Longitude,Latitude,datetime,UHI Index,location,rounded_time,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2],B01,B02,B03,B04,B06,B08,B11,B12,lwir11
0,-73.919037,40.814292,2021-07-24 15:53:00,1.034616,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621,1561.0,1390.0,1646.0,1686.0,1831.0,1854.0,2296.0,2194.0,38.681055
1,-73.918978,40.814365,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621,1561.0,1390.0,1646.0,1686.0,1831.0,1854.0,2296.0,2194.0,38.681055
2,-73.918927,40.814433,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621,1502.0,1214.0,1184.0,1408.0,1755.0,1510.0,2168.0,2089.0,38.681055
3,-73.918875,40.814500,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621,1566.0,1286.0,1570.0,1656.0,2169.0,1832.0,2319.0,2022.0,38.687891
4,-73.918827,40.814560,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621,1566.0,1338.0,1390.0,1492.0,1648.0,1810.0,1890.0,1659.0,38.404195


## Feature Engineering

In [10]:
# Calculate NDVI (Normalized Difference Vegetation Index) and handle division by zero by replacing infinities with NaN.
# See the Sentinel-2 sample notebook for more information about the NDVI index
uhi_data['NDVI'] = (uhi_data['B08'] - uhi_data['B04']) / (uhi_data['B08'] + uhi_data['B04'])
uhi_data['NDVI'] = uhi_data['NDVI'].replace([np.inf, -np.inf], np.nan) 

In [11]:
# Compute EVI (Enhanced Vegetation Index)
uhi_data['EVI'] = 2.5 * (uhi_data['B08'] - uhi_data['B04']) / (uhi_data['B08'] + 6 * uhi_data['B04'] - 7.5 * uhi_data['B02'] + 1)
uhi_data['EVI'] = uhi_data['EVI'].replace([np.inf, -np.inf], np.nan)
uhi_data['EVI'].fillna(uhi_data['EVI'].median(), inplace=True)

In [12]:
# Compute NDBI
uhi_data['NDBI'] = (uhi_data['B11'] - uhi_data['B08']) / (uhi_data['B11'] + uhi_data['B08'])
uhi_data['NDBI'] = uhi_data['NDBI'].replace([np.inf, -np.inf], np.nan)

In [13]:
# Compute NDWI (Water Index)
uhi_data['NDWI'] = (uhi_data['B03'] - uhi_data['B08']) / (uhi_data['B03'] + uhi_data['B08'])
uhi_data['NDWI'] = uhi_data['NDWI'].replace([np.inf, -np.inf], np.nan)

In [14]:
# Albeto from sentinel2
uhi_data['Albedo'] = (0.356 * uhi_data['B02'] + 0.130 * uhi_data['B03'] + 0.373 * uhi_data['B04'] + 0.085 * uhi_data['B08'] + 0.072 * uhi_data['B11'] + 0.018 * uhi_data['B12'])
uhi_data['Albedo'] = uhi_data['Albedo'].replace([np.inf, -np.inf], np.nan)

In [15]:
uhi_data["temp_gradient"] = uhi_data["lwir11"] - uhi_data["Air Temp at Surface [degC]"]
uhi_data['temp_gradient'] = uhi_data['temp_gradient'].replace([np.inf, -np.inf], np.nan)

In [16]:
uhi_data["humidity_temp_interaction"] = uhi_data["Relative Humidity [percent]"] * uhi_data["Air Temp at Surface [degC]"]
uhi_data['humidity_temp_interaction'] = uhi_data['humidity_temp_interaction'].replace([np.inf, -np.inf], np.nan)

In [17]:
uhi_data["wind_temp_interaction"] = uhi_data["Avg Wind Speed [m/s]"] * uhi_data["Air Temp at Surface [degC]"]
uhi_data['wind_temp_interaction'] = uhi_data['wind_temp_interaction'].replace([np.inf, -np.inf], np.nan)

In [18]:
uhi_data["solar_temp_interaction"] = uhi_data["Solar Flux [W/m^2]"] * uhi_data["Air Temp at Surface [degC]"]
uhi_data['solar_temp_interaction'] = uhi_data['solar_temp_interaction'].replace([np.inf, -np.inf], np.nan)

In [19]:
uhi_data["humidity_wind_interaction"] = uhi_data["Relative Humidity [percent]"] * uhi_data["Avg Wind Speed [m/s]"]
uhi_data['humidity_wind_interaction'] = uhi_data['humidity_wind_interaction'].replace([np.inf, -np.inf], np.nan)

In [20]:
uhi_data["wind_chill"] = 13.12 + 0.6215 * uhi_data["Air Temp at Surface [degC]"] - 11.37 * uhi_data["Avg Wind Speed [m/s]"] ** 0.16 + 0.3965 * uhi_data["Air Temp at Surface [degC]"] * uhi_data["Avg Wind Speed [m/s]"] ** 0.16
uhi_data['wind_chill'] = uhi_data['wind_chill'].replace([np.inf, -np.inf], np.nan)


In [21]:
uhi_data["heat_index"] = -42.379 + 2.049 * uhi_data["Air Temp at Surface [degC]"] + 10.143 * uhi_data["Relative Humidity [percent]"] - 0.224 * uhi_data["Air Temp at Surface [degC]"] * uhi_data["Relative Humidity [percent]"]
uhi_data['heat_index'] = uhi_data['heat_index'].replace([np.inf, -np.inf], np.nan)

In [22]:
# Display the first few rows of results
print(uhi_data.head())

   Longitude   Latitude             datetime  UHI Index location  \
0 -73.919037  40.814292  2021-07-24 15:53:00   1.034616    Bronx   
1 -73.918978  40.814365  2021-07-24 15:53:00   1.028125    Bronx   
2 -73.918927  40.814433  2021-07-24 15:53:00   1.028125    Bronx   
3 -73.918875  40.814500  2021-07-24 15:53:00   1.025961    Bronx   
4 -73.918827  40.814560  2021-07-24 15:53:00   1.025961    Bronx   

          rounded_time  Air Temp at Surface [degC]  \
0  2021-07-24 15:55:00                        27.2   
1  2021-07-24 15:55:00                        27.2   
2  2021-07-24 15:55:00                        27.2   
3  2021-07-24 15:55:00                        27.2   
4  2021-07-24 15:55:00                        27.2   

   Relative Humidity [percent]  Avg Wind Speed [m/s]  \
0                         47.3                   2.6   
1                         47.3                   2.6   
2                         47.3                   2.6   
3                         47.3            

In [23]:
uhi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11269 entries, 0 to 11268
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Longitude                    11269 non-null  float64
 1   Latitude                     11269 non-null  float64
 2   datetime                     11269 non-null  object 
 3   UHI Index                    11269 non-null  float64
 4   location                     11269 non-null  object 
 5   rounded_time                 11269 non-null  object 
 6   Air Temp at Surface [degC]   11269 non-null  float64
 7   Relative Humidity [percent]  11269 non-null  float64
 8   Avg Wind Speed [m/s]         11269 non-null  float64
 9   Wind Direction [degrees]     11269 non-null  int64  
 10  Solar Flux [W/m^2]           11269 non-null  int64  
 11  B01                          11269 non-null  object 
 12  B02                          11269 non-null  object 
 13  B03             

## Removing duplicates & NaN
Identical or duplicate entries are removed based on specific columns, in our case [ 'B01','B04','B06','B08','NDVI']. This ensures that the dataset is not biased or skewed by repetitive data, allowing the model to train on unique, relevant observations.

In [24]:
# Check for NaN values in the entire DataFrame
print(uhi_data.isna().sum())

Longitude                      0
Latitude                       0
datetime                       0
UHI Index                      0
location                       0
rounded_time                   0
Air Temp at Surface [degC]     0
Relative Humidity [percent]    0
Avg Wind Speed [m/s]           0
Wind Direction [degrees]       0
Solar Flux [W/m^2]             0
B01                            0
B02                            0
B03                            0
B04                            0
B06                            0
B08                            0
B11                            0
B12                            0
lwir11                         0
NDVI                           0
EVI                            0
NDBI                           0
NDWI                           0
Albedo                         0
temp_gradient                  0
humidity_temp_interaction      0
wind_temp_interaction          0
solar_temp_interaction         0
humidity_wind_interaction      0
wind_chill

In [25]:
# Remove duplicate rows from the DataFrame based on specified columns and keep the first occurrence
columns_to_check = uhi_data.columns.to_list()

for col in columns_to_check:
    # Check if the value is a numpy array and has more than one dimension
    uhi_data[col] = uhi_data[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)

# Now remove duplicates
uhi_data = uhi_data.drop_duplicates(subset=columns_to_check, keep='first')
uhi_data.head()


,Longitude,Latitude,datetime,UHI Index,location,rounded_time,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],...,NDBI,NDWI,Albedo,temp_gradient,humidity_temp_interaction,wind_temp_interaction,solar_temp_interaction,humidity_wind_interaction,wind_chill,heat_index
0,-73.919037,40.814292,2021-07-24 15:53:00,1.034616,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,...,0.106506,-0.059429,1700.092,11.481055,1286.56,70.72,16891.2,122.98,29.342932,204.92826
1,-73.918978,40.814365,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,...,0.106506,-0.059429,1700.092,11.481055,1286.56,70.72,16891.2,122.98,29.342932,204.92826
2,-73.918927,40.814433,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,...,0.178902,-0.121010,1433.336,11.481055,1286.56,70.72,16891.2,122.98,29.342932,204.92826
3,-73.918875,40.814500,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,...,0.117321,-0.077014,1638.688,11.487891,1286.56,70.72,16891.2,122.98,29.342932,204.92826
4,-73.918827,40.814560,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,...,0.021622,-0.131250,1533.336,11.204195,1286.56,70.72,16891.2,122.98,29.342932,204.92826


In [26]:
# Resetting the index of the dataset
uhi_data=uhi_data.reset_index(drop=True)

In [27]:
uhi_data.to_csv("uhi_data.csv", index=False)

## Model Building

<p align="justify"> Now let us select the columns required for our model building exercise. We will consider only Band B01, Band B06 and NDVI from the Sentinel-2 data as our predictor variables. It does not make sense to use latitude and longitude as predictor variables, as they do not have any direct impact on predicting the UHI index.</p>


In [28]:
# Retaining only the columns for B01, B06, NDVI, and UHI Index in the dataset.
uhi_data = uhi_data[['B01','B06', 'UHI Index', 'lwir11', 'temp_gradient', 'humidity_temp_interaction', 'wind_temp_interaction', 'solar_temp_interaction', 'humidity_wind_interaction', 'wind_chill', 'heat_index', 'Air Temp at Surface [degC]','Relative Humidity [percent]','Avg Wind Speed [m/s]','Wind Direction [degrees]', 'Solar Flux [W/m^2]']]

### Train and Test Split 

<p align="justify">We will now split the data into 70% training data and 30% test data. Scikit-learn alias “sklearn” is a robust library for machine learning in Python. The scikit-learn library has a <i><b>model_selection</b></i> module in which there is a splitting function <i><b>train_test_split</b></i>. You can use the same.</p>

In [34]:
# Split the data into features (X) and target (y), and then into training and testing sets
X = uhi_data.drop(columns=['UHI Index']).values
y = uhi_data ['UHI Index'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=123)

### Feature Scaling 

<p align="justify"> Before initiating the model training we may have to execute different data pre-processing steps. Here we are using Standard Scaler.</p>

<p align = "justify">Feature Scaling is a data preprocessing step for numerical features. Many machine learning algorithms like Gradient descent methods, KNN algorithm, linear and logistic regression, etc. require data scaling to produce good results. Scikit learn provides functions that can be used to apply data scaling. Here we are using Standard Scaler. The idea behind Standard Scaler is that it will transform your data such that its distribution will have a mean value 0 and standard deviation of 1.</p>

In [35]:
# Scale the training and test data using standardscaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Model Training

## Hypertuning for over-fitting

In [36]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Define the hyperparameter search space
param_dist = {
    'n_estimators': randint(50, 500),         # Number of trees in the forest
    'max_depth': randint(1, 30),              # Maximum depth of each tree
    'min_samples_split': randint(2, 10),      # Minimum number of samples required to split a node
    'min_samples_leaf': randint(1, 10),       # Minimum number of samples at a leaf node
    'max_features': ['auto', 'sqrt'], # Number of features to consider for splitting
    'bootstrap': [True, False]                # Whether to use bootstrap samples
}

# Create a random forest regressor
rf = RandomForestRegressor(random_state=42)

# Perform randomized search with cross-validation
rand_search = RandomizedSearchCV(
    estimator=rf, 
    param_distributions=param_dist, 
    n_iter=100,  # Number of random combinations to try
    cv=5,       # 3-fold cross-validation
    verbose=2, 
    n_jobs=-1,  # Use all available CPU cores
    random_state=42
)

# Fit the random search model
rand_search.fit(X_train, y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=8, min_samples_split=6, n_estimators=70; total time=   0.0s
[CV] END bootstrap=True, max_depth=26, max_features=auto, min_samples_leaf=7, min_samples_split=4, n_estimators=137; total time=   0.0s
[CV] END bootstrap=True, max_depth=26, max_features=auto, min_samples_leaf=7, min_samples_split=4, n_estimators=137; total time=   0.0s
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=8, min_samples_split=6, n_estimators=70; total time=   0.0s
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=8, min_samples_split=6, n_estimators=70; total time=   0.0s
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=8, min_samples_split=6, n_estimators=70; total time=   0.0s
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=8, min_samples_split=6, n_estimators=70; total tim

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x172647d90>,
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x301745650>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x301746490>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x173887dd0>},
                   random_state=42, verbose=2)

In [ ]:
# Create variable for the best model
best_rf = rand_search.best_estimator_
print('Best hyperparameters:',  rand_search.best_params_)

Best hyperparameters: {'bootstrap': False, 'max_depth': 23, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 322}


In [ ]:
# Train the Random Forest model on the training data
model = best_rf
model.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, max_depth=23, max_features='sqrt',
                      min_samples_leaf=2, min_samples_split=3, n_estimators=322,
                      random_state=42)

## Model Evaluation

<p align="justify">In this section, we make predictions on the training set and store them in the <b><i>insample_predictions</i></b> variable. The R² score is then calculated to gauge the model's performance on the training data.</p>


In [40]:
# Make predictions on the training data
insample_predictions = model.predict(X_train)

In [41]:
# calculate R-squared score for in-sample predictions
Y_train = y_train.tolist()
r2_score(Y_train, insample_predictions)

0.9744627779367939

### Out-Sample Evaluation

When evaluating a machine learning model, it is essential to correctly and fairly evaluate the model's ability to generalize. This is because models have a tendency to overfit the dataset they are trained on. To estimate the out-of-sample performance, we will predict on the test data now. 

In [42]:
# Make predictions on the test data
outsample_predictions = model.predict(X_test)

In [43]:
# calculate R-squared score for out-sample predictions
Y_test = y_test.tolist()
r2_score(Y_test, outsample_predictions)

0.8706786006976708

## Submission

In [45]:
#Reading the coordinates for the submission
test_file = pd.read_csv('Submission_template.csv')
test_file.head()

,Longitude,Latitude,UHI Index
0,-73.971665,40.788763,NaN
1,-73.971928,40.788875,NaN
2,-73.967080,40.789080,NaN
3,-73.972550,40.789082,NaN
4,-73.969697,40.787953,NaN


In [46]:
# Load the training data from csv file and display the first few rows to inspect the data
weather_df = pd.read_csv("Training_data_uhi_index_weather.csv")
weather_df.head()

,Longitude,Latitude,datetime,UHI Index,location,rounded_time,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,-73.919037,40.814292,2021-07-24 15:53:00,1.034616,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
1,-73.918978,40.814365,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
2,-73.918927,40.814433,2021-07-24 15:53:00,1.028125,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
3,-73.918875,40.814500,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621
4,-73.918827,40.814560,2021-07-24 15:53:00,1.025961,Bronx,2021-07-24 15:55:00,27.2,47.3,2.6,165,621


In [47]:
weather_df.drop(["datetime", "UHI Index", "location", "rounded_time"], axis=1, inplace=True)
weather_df.head()

,Longitude,Latitude,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,-73.919037,40.814292,27.2,47.3,2.6,165,621
1,-73.918978,40.814365,27.2,47.3,2.6,165,621
2,-73.918927,40.814433,27.2,47.3,2.6,165,621
3,-73.918875,40.814500,27.2,47.3,2.6,165,621
4,-73.918827,40.814560,27.2,47.3,2.6,165,621


In [48]:
from shapely.geometry import Point
# Convert weather_df into a GeoDataFrame
weather_df["geometry"] = weather_df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
weather_density_df = gpd.GeoDataFrame(weather_df, geometry="geometry", crs="EPSG:4326")  # Ensure correct coordinate system

In [49]:
from shapely.geometry import Point
# Convert test_file into a GeoDataFrame
test_file["geometry"] = test_file.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
test_file_gdf = gpd.GeoDataFrame(test_file, geometry="geometry", crs="EPSG:4326")  # Ensure correct coordinate system

In [50]:
# Spatial nearest join: Add building data to uhi dataset
submission_data = gpd.sjoin_nearest(test_file_gdf, weather_density_df, how="left", distance_col="distances")
submission_data.head()

,Longitude_left,Latitude_left,UHI Index,geometry,index_right,Longitude_right,Latitude_right,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2],distances
0,-73.971665,40.788763,NaN,POINT (-73.97166 40.78876),4837,-73.971665,40.788763,26.6,50.5,3.1,154,584,0.0
1,-73.971928,40.788875,NaN,POINT (-73.97193 40.78888),4840,-73.971928,40.788875,26.6,50.5,3.1,154,584,0.0
2,-73.967080,40.789080,NaN,POINT (-73.96708 40.78908),4797,-73.967080,40.789080,26.3,50.9,3.0,158,219,0.0
3,-73.972550,40.789082,NaN,POINT (-73.97255 40.78908),4848,-73.972550,40.789082,26.6,50.5,3.1,154,584,0.0
4,-73.969697,40.787953,NaN,POINT (-73.9697 40.78795),4819,-73.969697,40.787953,26.6,50.5,3.1,154,584,0.0


In [51]:
# Check for duplicates based on the 'geometry' column
duplicates = submission_data[submission_data.duplicated(subset=["geometry"], keep=False)]
print(duplicates)

     Longitude_left  Latitude_left  UHI Index                    geometry  \
89       -73.953007      40.768135        NaN  POINT (-73.95301 40.76814)   
89       -73.953007      40.768135        NaN  POINT (-73.95301 40.76814)   
457      -73.917182      40.833252        NaN  POINT (-73.91718 40.83325)   
457      -73.917182      40.833252        NaN  POINT (-73.91718 40.83325)   

     index_right  Longitude_right  Latitude_right  Air Temp at Surface [degC]  \
89          1470       -73.953007       40.768135                        26.6   
89          1471       -73.953007       40.768135                        26.6   
457         3074       -73.917182       40.833252                        27.2   
457         3075       -73.917182       40.833252                        27.2   

     Relative Humidity [percent]  Avg Wind Speed [m/s]  \
89                          48.3                   3.5   
89                          48.3                   3.5   
457                         47.8  

In [52]:
# Deduplicate by keeping the closest match
submission_data = submission_data.sort_values(by="distances").drop_duplicates(subset=["geometry"], keep="first")
submission_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1040 entries, 0 to 803
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Longitude_left               1040 non-null   float64 
 1   Latitude_left                1040 non-null   float64 
 2   UHI Index                    0 non-null      float64 
 3   geometry                     1040 non-null   geometry
 4   index_right                  1040 non-null   int64   
 5   Longitude_right              1040 non-null   float64 
 6   Latitude_right               1040 non-null   float64 
 7   Air Temp at Surface [degC]   1040 non-null   float64 
 8   Relative Humidity [percent]  1040 non-null   float64 
 9   Avg Wind Speed [m/s]         1040 non-null   float64 
 10  Wind Direction [degrees]     1040 non-null   int64   
 11  Solar Flux [W/m^2]           1040 non-null   int64   
 12  distances                    1040 non-null   float64 
dtypes

In [53]:
submission_data = submission_data.drop(submission_data.columns[[0,1,2,3,4,5,6]], axis=1)
submission_data.head()


,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2],distances
0,26.6,50.5,3.1,154,584,0.0
643,26.3,50.9,3.0,158,219,0.0
644,27.2,47.8,3.2,157,491,0.0
645,27.3,45.4,3.8,202,349,0.0
646,27.3,45.4,3.8,202,349,0.0


In [54]:
submission_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1040 entries, 0 to 803
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Air Temp at Surface [degC]   1040 non-null   float64
 1   Relative Humidity [percent]  1040 non-null   float64
 2   Avg Wind Speed [m/s]         1040 non-null   float64
 3   Wind Direction [degrees]     1040 non-null   int64  
 4   Solar Flux [W/m^2]           1040 non-null   int64  
 5   distances                    1040 non-null   float64
dtypes: float64(4), int64(2)
memory usage: 56.9 KB


In [55]:
# Mapping satellite data for submission.
val_data = map_satellite_data('S2_sample.tiff', 'Landsat_LST.tiff', 'Submission_template.csv')

Extracting band values: 100%|██████████| 1040/1040 [00:03<00:00, 311.76it/s]


In [56]:
val_data.head()

,B01,B02,B03,B04,B06,B08,B11,B12,lwir11
0,811.0,459.0,617.0,432.0,2089.0,2502.0,1474.0,893.0,36.202990
1,1208.0,562.0,731.0,647.0,2076.0,2906.0,1751.0,1188.0,36.202990
2,899.0,955.0,1052.0,1188.0,995.0,1246.0,1101.0,763.0,36.069687
3,1193.0,1132.0,1364.0,1512.0,1939.0,1774.0,2521.0,2346.0,36.886594
4,1097.0,1506.0,1642.0,1688.0,2204.0,2834.0,2248.0,1848.0,34.500816


In [57]:
# Combining ground data and final data into a single dataset.
val_data = combine_two_datasets(val_data,submission_data)
val_data.head()

,B01,B02,B03,B04,B06,B08,B11,B12,lwir11,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2],distances
0,811.0,459.0,617.0,432.0,2089.0,2502.0,1474.0,893.0,36.202990,26.6,50.5,3.1,154,584,0.0
1,1208.0,562.0,731.0,647.0,2076.0,2906.0,1751.0,1188.0,36.202990,26.6,50.5,3.1,154,584,0.0
2,899.0,955.0,1052.0,1188.0,995.0,1246.0,1101.0,763.0,36.069687,26.3,50.9,3.0,158,219,0.0
3,1193.0,1132.0,1364.0,1512.0,1939.0,1774.0,2521.0,2346.0,36.886594,26.6,50.5,3.1,154,584,0.0
4,1097.0,1506.0,1642.0,1688.0,2204.0,2834.0,2248.0,1848.0,34.500816,26.6,50.5,3.1,154,584,0.0


In [58]:
val_data["temp_gradient"] = val_data["lwir11"] - val_data["Air Temp at Surface [degC]"]
val_data['temp_gradient'] = val_data['temp_gradient'].replace([np.inf, -np.inf], np.nan)

In [59]:
val_data["humidity_temp_interaction"] = val_data["Relative Humidity [percent]"] * val_data["Air Temp at Surface [degC]"]
val_data['humidity_temp_interaction'] = val_data['humidity_temp_interaction'].replace([np.inf, -np.inf], np.nan)

In [60]:
val_data["wind_temp_interaction"] = val_data["Avg Wind Speed [m/s]"] * uhi_data["Air Temp at Surface [degC]"]
val_data['wind_temp_interaction'] = val_data['wind_temp_interaction'].replace([np.inf, -np.inf], np.nan)

In [61]:
val_data["solar_temp_interaction"] = val_data["Solar Flux [W/m^2]"] * val_data["Air Temp at Surface [degC]"]
val_data['solar_temp_interaction'] = val_data['solar_temp_interaction'].replace([np.inf, -np.inf], np.nan)

In [62]:
val_data["humidity_wind_interaction"] = val_data["Relative Humidity [percent]"] * val_data["Avg Wind Speed [m/s]"]
val_data['humidity_wind_interaction'] = val_data['humidity_wind_interaction'].replace([np.inf, -np.inf], np.nan)

In [63]:
val_data["wind_chill"] = 13.12 + 0.6215 * val_data["Air Temp at Surface [degC]"] - 11.37 * val_data["Avg Wind Speed [m/s]"] ** 0.16 + 0.3965 * val_data["Air Temp at Surface [degC]"] * val_data["Avg Wind Speed [m/s]"] ** 0.16
val_data['wind_chill'] = val_data['wind_chill'].replace([np.inf, -np.inf], np.nan)


In [64]:
val_data["heat_index"] = -42.379 + 2.049 * val_data["Air Temp at Surface [degC]"] + 10.143 * val_data["Relative Humidity [percent]"] - 0.224 * val_data["Air Temp at Surface [degC]"] * val_data["Relative Humidity [percent]"]
val_data['heat_index'] = val_data['heat_index'].replace([np.inf, -np.inf], np.nan)

In [65]:
val_data.head()

,B01,B02,B03,B04,B06,B08,B11,B12,lwir11,Air Temp at Surface [degC],...,Wind Direction [degrees],Solar Flux [W/m^2],distances,temp_gradient,humidity_temp_interaction,wind_temp_interaction,solar_temp_interaction,humidity_wind_interaction,wind_chill,heat_index
0,811.0,459.0,617.0,432.0,2089.0,2502.0,1474.0,893.0,36.202990,26.6,...,154,584,0.0,9.602990,1343.30,84.32,15534.4,156.55,28.665460,223.44670
1,1208.0,562.0,731.0,647.0,2076.0,2906.0,1751.0,1188.0,36.202990,26.6,...,154,584,0.0,9.602990,1343.30,84.32,15534.4,156.55,28.665460,223.44670
2,899.0,955.0,1052.0,1188.0,995.0,1246.0,1101.0,763.0,36.069687,26.3,...,158,219,0.0,9.769687,1338.67,81.60,5759.7,152.70,28.342363,227.92632
3,1193.0,1132.0,1364.0,1512.0,1939.0,1774.0,2521.0,2346.0,36.886594,26.6,...,154,584,0.0,10.286594,1343.30,84.32,15534.4,156.55,28.665460,223.44670
4,1097.0,1506.0,1642.0,1688.0,2204.0,2834.0,2248.0,1848.0,34.500816,26.6,...,154,584,0.0,7.900816,1343.30,84.32,15534.4,156.55,28.665460,223.44670


In [66]:
# Extracting specific columns (B01, B06, NDVI, NDBI and NDWI) from the validation dataset
submission_val_data=val_data.loc[:,['B01','B06', 'lwir11', 'temp_gradient', 'humidity_temp_interaction', 'wind_temp_interaction', 'solar_temp_interaction', 'humidity_wind_interaction', 'wind_chill', 'heat_index', 'Air Temp at Surface [degC]','Relative Humidity [percent]','Avg Wind Speed [m/s]','Wind Direction [degrees]', 'Solar Flux [W/m^2]']]
submission_val_data.head()

,B01,B06,lwir11,temp_gradient,humidity_temp_interaction,wind_temp_interaction,solar_temp_interaction,humidity_wind_interaction,wind_chill,heat_index,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,811.0,2089.0,36.202990,9.602990,1343.30,84.32,15534.4,156.55,28.665460,223.44670,26.6,50.5,3.1,154,584
1,1208.0,2076.0,36.202990,9.602990,1343.30,84.32,15534.4,156.55,28.665460,223.44670,26.6,50.5,3.1,154,584
2,899.0,995.0,36.069687,9.769687,1338.67,81.60,5759.7,152.70,28.342363,227.92632,26.3,50.9,3.0,158,219
3,1193.0,1939.0,36.886594,10.286594,1343.30,84.32,15534.4,156.55,28.665460,223.44670,26.6,50.5,3.1,154,584
4,1097.0,2204.0,34.500816,7.900816,1343.30,84.32,15534.4,156.55,28.665460,223.44670,26.6,50.5,3.1,154,584


In [67]:
submission_val_data = pd.DataFrame(submission_val_data)
submission_val_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1040 entries, 0 to 1039
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   B01                          1040 non-null   object 
 1   B06                          1040 non-null   object 
 2   lwir11                       1040 non-null   float64
 3   temp_gradient                1040 non-null   float64
 4   humidity_temp_interaction    1040 non-null   float64
 5   wind_temp_interaction        1040 non-null   float64
 6   solar_temp_interaction       1040 non-null   float64
 7   humidity_wind_interaction    1040 non-null   float64
 8   wind_chill                   1040 non-null   float64
 9   heat_index                   1040 non-null   float64
 10  Air Temp at Surface [degC]   1040 non-null   float64
 11  Relative Humidity [percent]  1040 non-null   float64
 12  Avg Wind Speed [m/s]         1040 non-null   float64
 13  Wind Direction [de

In [68]:
# Feature Scaling 
submission_val_data = submission_val_data.values
transformed_submission_data = sc.transform(submission_val_data)

In [69]:
#Making predictions
final_predictions = model.predict(transformed_submission_data)
final_prediction_series = pd.Series(final_predictions)

In [70]:
print(len(test_file['Longitude'].values))  # Length of longitude
print(len(test_file['Latitude'].values))   # Length of latitude
print(len(final_prediction_series.values)) # Length of UHI Index


1040
1040
1040


In [71]:
#Combining the results into dataframe
submission_df = pd.DataFrame({'Longitude':test_file['Longitude'].values, 'Latitude':test_file['Latitude'].values, 'UHI Index':final_prediction_series.values})

In [72]:
#Displaying the sample submission dataframe
submission_df.head()

,Longitude,Latitude,UHI Index
0,-73.971665,40.788763,0.963191
1,-73.971928,40.788875,0.963311
2,-73.967080,40.789080,0.967508
3,-73.972550,40.789082,0.967453
4,-73.969697,40.787953,0.963562


In [73]:
# Dumping the predictions into a csv file.
submission_df.to_csv("submission.csv",index = False)